# Market Cap

- load coinmarketcap data 
- create an crypto equivalent of "stocks" with marketcap, IPO, sector 



In [1]:
import time
import random
import pandas as pd
from tqdm import tqdm
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import numpy as np

In [2]:
# get top 100 coins from coingecko 

top_100_coins = cg.get_coins_markets(vs_currency='usd')

top_100_coins_df = pd.DataFrame(top_100_coins)
top_100_coins_df.set_index('id',inplace=True)
col_list = ['market_cap', ]
top_100_coins_df = top_100_coins_df.loc[:, col_list]

top_100_coins_list = top_100_coins_df.index.tolist()


top_100_coins_df

,market_cap
id,
bitcoin,851165633926
ethereum,408663307732
binancecoin,81684939616
ripple,77810326837
cardano,67995769831
...,...
icon,1263863907
pirate-chain,1234724958
paxos-standard,1232338318


In [3]:
# for each coin id get detailed information and store it into a market_df

market = []
raw_data = []
for idx in tqdm(top_100_coins_list):
    coin = cg.get_coin_by_id(id=idx, localization=False, vs_currencies='usd',include_market_cap=False,
                             include_24hr_vol=False,include_24hr_change=False,include_last_updated_at=False)
    raw_data.append(coin)
    coin_info = [idx]
    coin_info.append(coin['symbol'])
    coin_info.append(str(coin['name']))
    coin_info.append(coin['genesis_date'])
    coin_info.append(coin['market_cap_rank'])
    coin_info.append(str(coin['hashing_algorithm']))
    coin_info.append(coin['coingecko_rank'])
    coin_info.append(coin['coingecko_score'])
    coin_info.append(coin['developer_score'])
    coin_info.append(coin['community_score'])
    coin_info.append(coin['liquidity_score'])
    coin_info.append(coin['public_interest_score'])
    market.append(coin_info)
    time.sleep(random.randint(0, 5))
    
market_df = pd.DataFrame(market, columns = ['id', 'symbol', 'name', 'genesis_date',
                                            'market_cap_rank', 'hashing_algorithm', 
                                            'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 
                                            'liquidity_score', 'public_interest_score'])
    
market_df['genesis_date'] = pd.to_datetime(market_df['genesis_date'])
market_df['base'] = 'usd'

100%|██████████| 100/100 [06:58<00:00,  4.18s/it]


In [4]:
# combine the information into the market

market = top_100_coins_df.merge(market_df, on='id')

market.set_index(['symbol', 'base'], inplace=True)

In [5]:
# todo figure out why name and hasing algorithm are object and not string
market.info()
market['name'] = market['name'].astype(str)
market['hashing_algorithm'] = market['hashing_algorithm'].astype(str)
market.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100 entries, ('btc', 'usd') to ('tusd', 'usd')
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     100 non-null    object        
 1   market_cap             100 non-null    int64         
 2   name                   100 non-null    object        
 3   genesis_date           37 non-null     datetime64[ns]
 4   market_cap_rank        100 non-null    int64         
 5   hashing_algorithm      100 non-null    object        
 6   coingecko_rank         100 non-null    int64         
 7   coingecko_score        100 non-null    float64       
 8   developer_score        100 non-null    float64       
 9   community_score        100 non-null    float64       
 10  liquidity_score        100 non-null    float64       
 11  public_interest_score  100 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64

In [6]:
market.head(10)

,,id,market_cap,name,genesis_date,market_cap_rank,hashing_algorithm,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score
symbol,base,,,,,,,,,,,,
btc,usd,bitcoin,851165633926,Bitcoin,2009-01-03,1,SHA-256,2,81.149,98.874,74.606,100.084,0.0
eth,usd,ethereum,408663307732,Ethereum,2015-07-30,2,Ethash,3,78.085,97.194,65.231,99.843,0.0
bnb,usd,binancecoin,81684939616,Binance Coin,2017-07-08,3,None,5,67.430,73.243,66.596,80.872,0.0
xrp,usd,ripple,77810326837,XRP,NaT,4,None,9,65.219,71.122,54.329,86.490,0.0
ada,usd,cardano,67995769831,Cardano,NaT,5,None,6,66.641,70.441,61.873,86.307,0.0
doge,usd,dogecoin,64856353213,Dogecoin,2013-12-08,6,Scrypt,4,72.983,77.948,81.160,87.606,0.0
usdt,usd,tether,58412066592,Tether,NaT,7,None,156,41.764,0.000,10.712,106.774,0.0
dot,usd,polkadot,40717651579,Polkadot,NaT,8,None,139,43.098,0.000,42.842,79.419,0.0
bch,usd,bitcoin-cash,21680014041,Bitcoin Cash,NaT,9,SHA-256,118,44.919,0.000,47.911,85.319,0.0


In [7]:
market.index.tolist

<bound method IndexOpsMixin.tolist of MultiIndex([(     'btc', 'usd'),
            (     'eth', 'usd'),
            (     'bnb', 'usd'),
            (     'xrp', 'usd'),
            (     'ada', 'usd'),
            (    'doge', 'usd'),
            (    'usdt', 'usd'),
            (     'dot', 'usd'),
            (     'bch', 'usd'),
            (     'ltc', 'usd'),
            (     'uni', 'usd'),
            (    'link', 'usd'),
            (    'usdc', 'usd'),
            (     'xlm', 'usd'),
            (     'sol', 'usd'),
            (   'matic', 'usd'),
            (     'etc', 'usd'),
            (     'vet', 'usd'),
            (   'theta', 'usd'),
            (     'eos', 'usd'),
            (     'trx', 'usd'),
            (    'shib', 'usd'),
            (    'aave', 'usd'),
            (    'wbtc', 'usd'),
            (    'busd', 'usd'),
            (     'okb', 'usd'),
            (     'fil', 'usd'),
            (     'neo', 'usd'),
            (     'xmr', 'usd'),
     

## Create categorical data from raw data and store in seperate data frame

In [8]:
def categorical_dict_to_np(cat_dict, unique_cats):
    # create array of symbols and categoricals 
    cat_np = np.zeros((len(cat_dict), len(unique_cats)), dtype=int)
    for ii, [key, values] in enumerate(cat_dict.items()): 
        for value in values: 
            idx = np.where(unique_cats == value)
            # print("{} is on idx {}".format(value, idx))
            cat_np[ii][idx] = 1
        assert(np.sum(cat_np[ii]) == len(values))
    return cat_np


def create_cat_dict(raw_data): 
    cat_dict = {}
    cat_list = []
    symbols = []
    max_len_cat = 0
    for coin in raw_data:
        categories = coin['categories']
        symbol = coin['symbol']
        cat_dict[symbol] = coin['categories']
        cat_list.extend(categories)
        symbols.append(symbol)
        max_len_cat = max(max_len_cat, len(coin['categories']))
    
    unique_cats = np.unique(cat_list)

    return cat_dict, unique_cats, symbols

def categoricals_from_raw_coingecko(raw_data): 
    cat_dict, unique_cats, symbols = create_cat_dict(raw_data)
    
    cat_np = categorical_dict_to_np(cat_dict, unique_cats)
    
    cat_df = pd.DataFrame(cat_np, columns=unique_cats)
    cat_df['symbol'] = symbols
    cat_df['base'] = 'usd'

    cat_df.set_index(['symbol', 'base'], inplace=True)
    print(unique_cats)
    print(cat_dict)
    return cat_df
    
cat_df = categoricals_from_raw_coingecko(raw_data)
cat_df

['Analytics' 'Artificial Intelligence' 'Asset-backed Tokens'
 'Automated Market Maker (AMM)' 'Avalanche Ecosystem'
 'Binance Smart Chain Ecosystem' 'Business Platform' 'Business Services'
 'Centralized Exchange Token (CEX)' 'Communication' 'Compound Tokens'
 'Cosmos Ecosystem' 'Cryptocurrency' 'Decentralized Exchange Token (DEX)'
 'Decentralized Finance (DeFi)' 'Derivatives' 'Entertainment'
 'Eth 2.0 Staking' 'Exchange-based Tokens' 'Finance / Banking' 'Gaming'
 'Governance' 'Infrastructure' 'Internet of Things (IOT)'
 'Lending/Borrowing' 'Masternodes' 'Meme Tokens' 'Metaverse'
 'Near Protocol Ecosystem' 'Non-Fungible Tokens (NFT)' 'Oracle'
 'Polkadot Ecosystem' 'Polygon Ecosystem' 'Privacy Coins' 'Protocol'
 'Seigniorage' 'Smart Contract Platform' 'Solana Ecosystem' 'Sports'
 'Stablecoins' 'Storage' 'Synthetic Issuer' 'Terra Ecosystem'
 'Tokenized BTC' 'USD Stablecoin' 'Wrapped-Tokens' 'Yearn Ecosystem'
 'Yield Aggregator' 'Yield Farming']
{'btc': ['Cryptocurrency'], 'eth': ['Smart Co

,,Analytics,Artificial Intelligence,Asset-backed Tokens,Automated Market Maker (AMM),Avalanche Ecosystem,Binance Smart Chain Ecosystem,Business Platform,Business Services,Centralized Exchange Token (CEX),Communication,...,Stablecoins,Storage,Synthetic Issuer,Terra Ecosystem,Tokenized BTC,USD Stablecoin,Wrapped-Tokens,Yearn Ecosystem,Yield Aggregator,Yield Farming
symbol,base,,,,,,,,,,,,,,,,,,,,,
btc,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eth,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bnb,usd,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
xrp,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ada,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icx,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
arrr,usd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pax,usd,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [9]:
DATA_STORE = '../data/crypto.h5'
with pd.HDFStore(DATA_STORE) as store:
    store.put('coingecko/top100/market', market)
    store.put('coingecko/top100/cats', cat_df)
print("done")

done
